# Assignment-1: Computational Genomics
## Task 1.1
A dataset "dataset.h5ad" has been provided, which contains a single-cell gene expression matrix. The following are the tasks to be performed on this dataset. 
1. Quality control, normalization, feature selection and PCA.
2. Following this, data is to be clustered using the Leiden Clustering Algorithm. 
3. Differential expression analysis is to be performed upon each cluster to identify the marker genes.  
4. Based on the marker genes identified, the clusters obtained are to be annotated. 

Note - the dataset can be accessed from the following link: [Large-Files](https://drive.google.com/drive/folders/1e78HYxcL1Z_czrz-b37Mc8F3HIk6b6th?usp=sharing). Let us begin this task by importing the required packages. 

In [2]:
import scanpy as sc 
import pandas as pd
import numpy as np

Loading the dataset - 

In [4]:
matrix = sc.read('C:/Personal/Academics/IITK Resources/Sem-7/Computational-Genomics/Large-Files/dataset.h5ad')
matrix

AnnData object with n_obs × n_vars = 10727 × 12303

What does this output mean? 
- n_obs refers to the number of cells. Hence, here we have 10,727 cells. 
- n_vars refers to the number of genes. This matrix contains details about 12,303 genes. 